### This is the final notebook for *ALL* the emission calculations.

In [1]:
# import numpy as np
# class Emiss:
#     def __init__(self, nrows = 82, ncols = 132, df = 1):
#         self.lon = []
#         self.lat = []
#         self.x = []
#         self.y = []
#         self.prod = []
#         self.df = df
#         self.emi = np.zeros((5, nrows, ncols))

In [1]:
# %matplotlib inline
import matplotlib.pyplot as plt
import gridgen_class_v2 as gc
from gridgen_class_v2 import Emiss as Emiss
import numpy as np
import counties
from openpyxl import load_workbook
import bisect
from mpl_toolkits.basemap import Basemap
import pandas as pd
# from oil_gas_fields import fields as fields
from oil_gas_fields import fields
import shapefile
# from shapely.geometry import shape
# from shapely.geometry import Polygon
# from shapely.geometry.polygon import LinearRing
import fiona
from fer_loc import ferplant

mt = gc.Map(36)
m = mt.GenMap()
x0 = mt.x[0,:]
y0 = mt.y[:,0]
id_dict = mt.GenDic()

county1 = counties.county()
pos_dic = county1.county_dict()

bdwb = load_workbook('/Users/oulongwen/Google Drive/self share/LCA breakdown of lignocol electricity long trans.xlsx',
                     data_only = True, read_only = True)
bd = bdwb['Breakdown']
assum = bdwb['Assumptions']
total_elec = assum['H5'].value

fer_dict = ferplant.fer_dict
fertilizer = load_workbook('/Users/oulongwen/Google Drive/self share/fertilizer plants.xlsx',
                     data_only = True, read_only = True)

# phosp = fertilizer['P']
potas = fertilizer['K2O']
urea = fertilizer['Urea']
nacid = fertilizer['Nitric acid']
an = fertilizer['Ammonia nitrate']
nh3 = fertilizer['Ammonia']
sacid = fertilizer['Sulfuric acid']
prock = fertilizer['P rock']
pacid = fertilizer['P acid']
usolution = fertilizer['UAN']
asul = fertilizer['Ammonium sulfate']
dmap = fertilizer['DMAP']

idx = mt.idx
# mt.pipe()

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/openpyxl/workbook/names/named_range.py:125: UserWarning: Discarded range with reserved name
  warnings.warn("Discarded range with reserved name")


#### 0: PM2.5 (9), 1: VOC (5), 2: NOx (7), 3: SOx (10), 4: NH3 (15)

In [3]:
index = ['9', '5', '7', '10', '15']

In [4]:
# Corn farming emission

corn = Emiss()
wb = load_workbook('/Users/oulongwen/Downloads/2012 corn prod in bushel.xlsx',data_only=True, read_only = True)
ws = wb['corn']
fid = []
prod = []
for row in ws.iter_rows('T2:V'+str(ws.max_row)):
    try:
        float(row[0].value)
        fid.append(row[-1].value)
        prod.append(row[0].value)
    except:
        pass
    
for id, production in zip(fid, prod):
    if str(id) in pos_dic.keys():
        pin = pos_dic[str(id)]
        corn.lat.append(pin[0])
        corn.lon.append(pin[1])
        corn.prod.append(production)        
corn.x, corn.y = m(corn.lon, corn.lat)

# Calculate emission
ctotal = sum(corn.prod)
for xi, yi, cp in zip(corn.x, corn.y, corn.prod):
    xind=bisect.bisect_left(x0, xi)
    yind=bisect.bisect_left(y0, yi)
    if (yind<=mt.nrows)&(xind>=1):
        for n in range(5):
            corn.emi[n][yind-1][xind-1]+=cp/ctotal*total_elec*bd['F'+ index[n]].value/10**6 * corn.df
np.save('./Data/corn', corn.emi)
# m.pcolormesh(mt.x,mt.y,corn.emi[0],cmap='RdPu')
plt.savefig('/Users/oulongwen/Desktop/corn_farming_pm25', dpi = 500)
# plt.show()

# State corn production
state = pd.read_csv('/Users/oulongwen/Downloads/state_latlon.csv',index_col = 'FIPS')
state['prod'] = pd.Series(np.zeros(len(state.index)),index = state.index)
# statelat = []
# statelon = []
# stateprod = []
tempid = []
for id, production in zip(fid, prod):
#     if id not in tempid:
#         statelat.append(state.ix[int(id[:2]),'latitude'])
#         statelon.append(state.ix[int(id[:2]),'longitude'])
#         tempid.append(id)
    state.ix[int(id[:2]),'prod'] += production
state = state.drop(state.index[[0,11]])     # remove HI and AK
statecorn = Emiss()
statecorn.lon = np.array(state['longitude'])
statecorn.lat = np.array(state['latitude'])
statecorn.prod = np.array(state['prod'])
statecorn.x, statecorn.y = m(statecorn.lon, statecorn.lat)

In [4]:
# Crude oil extraction emission
# from oil_gas_fields import fields as fields
# For initialization 
# emission = np.zeros((mt.nrows,mt.ncols))

oilex = Emiss(df = 0.475)

oilfids = fields.oilfid
oilprods = fields.oilprod

# oillat=[]
# oillon=[]
# oilprod=[]
for id, production in zip(oilfids, oilprods):
    if str(id) in pos_dic.keys():
        pin = pos_dic[str(id)]
        oilex.lat.append(pin[0])
        oilex.lon.append(pin[1])
        oilex.prod.append(production)
        
oilex.x,oilex.y = m(oilex.lon, oilex.lat)

oiltotal = sum(oilex.prod)
for xi, yi, oil in zip(oilex.x, oilex.y, oilex.prod):
    xind=bisect.bisect_left(x0, xi)
    yind=bisect.bisect_left(y0, yi)
    if (yind<=mt.nrows)&(xind>=1):
        for n in range(5):
            oilex.emi[n][yind-1][xind-1]+=oil/oiltotal*total_elec*bd['G'+index[n]].value/10**6 * oilex.df
np.save('./Data/oilex', oilex.emi)
# m.pcolormesh(mt.x,mt.y,emission,cmap='RdPu')
# plt.show()

In [5]:
# NG extraction emission

# For initialization 
# emission = np.zeros((mt.nrows,mt.ncols))
gas = Emiss(df = 0.835)

gasfids = fields.gasfid
gasprods = fields.gasprod

# gaslat=[]
# gaslon=[]
# gasprod=[]
for id, production in zip(gasfids, gasprods):
    if str(id) in pos_dic.keys():
        pin = pos_dic[str(id)]
        gas.lat.append(pin[0])
        gas.lon.append(pin[1])
        gas.prod.append(production)
        
gas.x, gas.y = m(gas.lon, gas.lat)

gastotal = sum(gas.prod)
for xi, yi, gasp in zip(gas.x, gas.y, gas.prod):
    xind=bisect.bisect_left(x0, xi)
    yind=bisect.bisect_left(y0, yi)
    if (yind<=mt.nrows)&(xind>=1):
        for n in range(5):
            gas.emi[n][yind-1][xind-1]+=gasp/gastotal*total_elec*bd['N'+index[n]].value/10**6 * gas.df
np.save('./Data/gas', gas.emi)   
# m.pcolormesh(mt.x,mt.y,emission,cmap='RdPu')
# plt.show()

In [ ]:
# NG processing emission

# For initialization 
# emission = np.zeros((mt.nrows,mt.ncols))

# import shapefile

gaspr = Emiss(df = 0.835)
m.readshapefile('/Users/oulongwen/Downloads/NaturalGas_ProcessingPlants_US_EIA/NaturalGas_ProcessingPlants_US_2014_v2','ngproc')
# print(*m.ngproc[0])
# print(type(m.ngproc_info[1]))
ngtotal = 0
for info in m.ngproc_info:
    ngtotal += float(info['Capa_MMcfd'])
for info,ngproc in zip(m.ngproc_info, m.ngproc):
    xind=bisect.bisect_left(x0, ngproc[0])
    yind=bisect.bisect_left(y0, ngproc[1])
    if (yind<=mt.nrows)&(xind>=1):
        for n in range(5):
            gaspr.emi[n][yind-1][xind-1]+=float(info['Capa_MMcfd'])/ngtotal*total_elec*bd['O'+index[n]].value/10**6 * gaspr.df
np.save('./Data/gaspr', gaspr.emi)
# !!!!When using plt.plot or m.plot, the first and second parameter are actually the x and y coordinates of the points in map.
#     plt.plot(ngproc[0], ngproc[1], marker='o')

# m.pcolormesh(mt.x,mt.y,emission,cmap='RdPu')
# plt.show()

In [ ]:
# Emission from lignocol electricity generation

# Initialize emission
# emission = np.zeros((mt.nrows,mt.ncols))

ligno = Emiss()
m.readshapefile('/Users/oulongwen/Documents/Research/LCA air quality/Code/powerplants_coal_8_26_14/powerplants_coal_8_26_14','power')

coaltotal = 0
for info in m.power_info:
    coaltotal += float(info['namepcap'])
# Calculate emission
for info,power in zip(m.power_info, m.power):
    xind=bisect.bisect_left(x0, power[0])
    yind=bisect.bisect_left(y0, power[1])
    if (yind<=mt.nrows)&(xind>=1):
        for n in range(5):
            ligno.emi[n][yind-1][xind-1]+=float(info['namepcap'])/coaltotal*total_elec*bd['I'+index[n]].value/10**6 * ligno.df
np.save('./Data/ligno', ligno.emi)    

# m.pcolormesh(mt.x,mt.y,emission,cmap='RdPu')
# plt.show()

In [ ]:
# Emission from petroleum refining

# Initialize emission
# emission = np.zeros((mt.nrows,mt.ncols))
# m.readshapefile('/Users/oulongwen/Downloads/oil refineries/Canada_US_Refineries_Capacity','oilref')

ref = Emiss()
wbref = load_workbook('/Users/oulongwen/Downloads/oil refineries/Canada_US_Refineries_Capacity.xlsx',data_only=True, read_only = True)
wsref = wbref['refineries']

for row in wsref.iter_rows('A2:K'+str(ws.max_row)):
        ref.prod.append(row[4].value)                # capacity
        ref.lat.append(row[-2].value)
        ref.lon.append(row[-3].value)

# reftotal = sum(refprod)
ref.x, ref.y = m(ref.lon, ref.lat)
# for info in m.oilref_info:
#     oilreftotal += float(info['Capacity'])
# Calculate emission
reftotal = 0
for xi, yi, refp in zip(ref.x, ref.y, ref.prod):
    xind=bisect.bisect_left(x0, xi)
    yind=bisect.bisect_left(y0, yi)
    if (yind<=mt.nrows)&(xind>=1)&(xind<=mt.ncols):
        reftotal += refp
        
for xi, yi, refp in zip(ref.x, ref.y, ref.prod):
    xind=bisect.bisect_left(x0, xi)
    yind=bisect.bisect_left(y0, yi)
    if (yind<=mt.nrows)&(xind>=1)&(xind<=mt.ncols):
        for n in range(5):
            ref.emi[n][yind-1][xind-1]+=refp/reftotal*total_elec*bd['H'+index[n]].value/10**6 * ref.df
np.save('./Data/ref', ref.emi)    
# m.scatter(xref, yref, marker='o')
# m.pcolormesh(mt.x,mt.y,emission,cmap='RdPu')
# plt.show()

In [ ]:
# Emission from oil pipe

# mt.pipe()

dcru = pd.read_csv('/Users/oulongwen/Desktop/pipeline2.csv')
doil = dcru.ix[(dcru['naics'] == 486110) & (dcru['design_capacity'] > 0),['longitude','latitude','design_capacity','design_capacity_units']]
doil.drop_duplicates(inplace = True)
conv = 393.014779  # 1 MMBtu/hr = 393 HP
doil.loc[doil['design_capacity_units']=='E6BTU/HR','design_capacity'] *= conv
doil['design_capacity_units']='HP'
do2 = doil.round({'longitude':2, 'latitude':2})
do2 = do2.drop_duplicates()
doil = doil.loc[do2.index,['longitude','latitude', 'design_capacity', 'design_capacity_units']]
doil = doil[['longitude', 'latitude', 'design_capacity']]
doil = doil.ix[681:]          #delete first row whose capacity is 1
npdoil = np.array(doil)

oiltr = Emiss(df = 0.475)

oiltr.lon = npdoil[:,0]
oiltr.lat = npdoil[:,1]
oiltr.prod = npdoil[:,2]
oiltr.x, oiltr.y = m(oiltr.lon, oiltr.lat)
oiltrtotal = sum(oiltr.prod)
for xi, yi, oiltrp in zip(oiltr.x, oiltr.y, oiltr.prod):
    xind=bisect.bisect_left(x0, xi)
    yind=bisect.bisect_left(y0, yi)
    if (yind<=mt.nrows)&(xind>=1)&(xind<=mt.ncols)&(yind>=1):
        for n in range(5):
            oiltr.emi[n][yind-1][xind-1]+=oiltrp/oiltrtotal*total_elec*bd['K'+ index[n]].value/10**6 * oiltr.df
# for n in range(5):
#     oiltr.emi[n] += mt.oilpipe / mt.total_oilpipe * total_elec * bd['K'+index[n]].value / 10**6 * oiltr.df  
np.save('./Data/oiltr', oiltr.emi)

In [ ]:
# Emission from NG pipe

dng = pd.read_csv('/Users/oulongwen/Desktop/pipeline2.csv')
dgas = dng.ix[(dng['naics'] == 486210) & (dng['design_capacity'] > 0),['longitude','latitude','design_capacity','design_capacity_units']]
dgas.drop_duplicates(inplace = True)
conv = 393.014779
dgas.loc[dgas['design_capacity_units']=='E6BTU/HR','design_capacity']*=conv
dgas['design_capacity_units']='HP'
dp2 = dgas.round({'longitude':2, 'latitude':2})
dp2 = dp2.drop_duplicates()
dgas = dgas.loc[dp2.index, ['longitude','latitude','design_capacity','design_capacity_units']]
dgas = dgas[['longitude', 'latitude', 'design_capacity']]
npdgas = np.array(dgas)

ngtr = Emiss(df = 0.835)

ngtr.lon = npdgas[:,0]
ngtr.lat = npdgas[:,1]
ngtr.prod = npdgas[:,2]
ngtr.x, ngtr.y = m(ngtr.lon, ngtr.lat)
ngtrtotal = sum(ngtr.prod)
for xi, yi, ngtrp in zip(ngtr.x, ngtr.y, ngtr.prod):
    xind=bisect.bisect_left(x0, xi)
    yind=bisect.bisect_left(y0, yi)
    if (yind<=mt.nrows)&(xind>=1)&(xind<=mt.ncols)&(yind>=1):
        for n in range(5):
            ngtr.emi[n][yind-1][xind-1]+=ngtrp/ngtrtotal*total_elec*bd['P'+ index[n]].value/10**6 * ngtr.df
# for n in range(5):
#     ngtr.emi[n] += mt.ngpipe / mt.total_ngpipe * total_elec * bd['P'+index[n]].value / 10**6 * ngtr.df
np.save('./Data/ngtr', ngtr.emi)

In [ ]:
# VMT
vmtdata = pd.read_csv('/Users/oulongwen/Downloads/total_vmt.csv',index_col = 'ID')
vmtdata.index = vmtdata.index.map("{:05}".format)
vmtdata['lon'] = pd.Series(np.zeros(len(vmtdata.index)),index = vmtdata.index)
vmtdata['lat'] = pd.Series(np.zeros(len(vmtdata.index)),index = vmtdata.index)
for i in range(len(vmtdata.index)):
    if str(vmtdata.index[i]) in pos_dic.keys():
        pin = pos_dic[str(vmtdata.index[i])]
        vmtdata.ix[i,['lat', 'lon']] = pin
# totalvmt = vmtdata['total_vmt'].sum()
vmt = np.array(vmtdata[['lon','lat','total_vmt']])

In [ ]:
# Emission from diesel pipe

# Initialize emission
# emission = np.zeros((mt.nrows,mt.ncols))

dieseltr = Emiss()
# for n in range(5):
#     dieseltr.emi[n] += mt.dieselpipe / mt.total_dieselpipe * total_elec * bd['L'+index[n]].value / 10**6 * dieseltr.df
dieseltr.lon = vmt[:,0]
dieseltr.lat = vmt[:,1]
dieseltr.prod = vmt[:,2]
dieseltr.x, dieseltr.y = m(dieseltr.lon, dieseltr.lat)
dieseltrtotal = sum(dieseltr.prod)
for xi, yi, dtrp in zip(dieseltr.x, dieseltr.y, dieseltr.prod):
    xind=bisect.bisect_left(x0, xi)
    yind=bisect.bisect_left(y0, yi)
    if (yind<=mt.nrows)&(xind>=1)&(xind<=mt.ncols)&(yind>=1):
        for n in range(5):
            dieseltr.emi[n][yind-1][xind-1]+=dtrp/dieseltrtotal*total_elec*bd['L'+ index[n]].value/10**6 * dieseltr.df
np.save('./Data/dieseltr', dieseltr.emi)

In [ ]:
# K2O fertilzer emission

# Initialize emission
# emission = np.zeros((mt.nrows,mt.ncols))

k = Emiss(df = 0.17)
kloc = []
# k.lon = []
# k.lat = []
# k.prod = []
for row in potas.iter_rows('A2:G'+ str(potas.max_row)):
    name = row[0].value
    if name != None:
        name = name.split(',')[0]
        if 'total' not in name.lower():
            if name.lower() in fer_dict.keys():
                try:
                    k.prod.append(float(row[-1].value))
                    kloc.append(name)
                    lon, lat = fer_dict[name.lower()]
                    k.lon.append(lon)
                    k.lat.append(lat)
                except:
                    pass
                
k.x, k.y = m(k.lon, k.lat)

# Calculate emission
ktotal = sum(k.prod)
for xi, yi, kp in zip(k.x, k.y, k.prod):
    xind=bisect.bisect_left(x0, xi)
    yind=bisect.bisect_left(y0, yi)
    if (yind<=mt.nrows)&(xind>=1):
        for n in range(5):
            k.emi[n][yind-1][xind-1]+=kp/ktotal*total_elec*bd['E'+index[0]].value/10**6 * k.df
np.save('./Data/k', k.emi)

In [ ]:
# Urea emission

u = Emiss(df = 0.565)
uloc = []
# u.lon = []
# u.lat = []
# u.prod = []

for row in urea.iter_rows('A2:G'+ str(urea.max_row)):
    name = row[0].value
    if name != None:
        name = name.split(',')[0]
        if 'total' not in name.lower():
            if name.lower() in fer_dict.keys():
                try:
                    u.prod.append(float(row[-1].value))
                    uloc.append(name)
                    lon, lat = fer_dict[name.lower()]
                    u.lon.append(lon)
                    u.lat.append(lat)
                except:
                    pass
                
u.x, u.y = m(u.lon, u.lat)

# Calculate emission
utotal = sum(u.prod)
for xi, yi, up in zip(u.x, u.y, u.prod):
    xind=bisect.bisect_left(x0, xi)
    yind=bisect.bisect_left(y0, yi)
    if (yind<=mt.nrows)&(xind>=1):
        for n in range(5):
            u.emi[n][yind-1][xind-1]+=up/utotal*total_elec*(bd['R'+index[n]].value+bd['AP'+index[n]].value)/10**6 * u.df
np.save('./Data/u', u.emi)

In [ ]:
# Sulfuric acid emission

sa = Emiss(df = 0.720)
# sa.lon = []
# sa.lat = []
# sa.prod = []

saloc = []

for row in sacid.iter_rows('A2:B40'):
    loc = row[0].value
    if loc != None:
        location = loc.split(';')
#         sprod = row[-1].value/len(location)
        for i, name in enumerate(location):
            if i == 0:
                name, state = name.split(',')[1:3]
            else:
                name, state = name.split(',')[0:2]
            name = name.strip()
            if 'total' not in name.lower():
                if name.lower() in fer_dict.keys():
                    try:
                        sa.prod.append(float(row[-1].value/len(location)))
                        saloc.append(name)
                        lon, lat = fer_dict[name.lower()]
                        sa.lon.append(lon)
                        sa.lat.append(lat)
                    except:
                        pass
                else:
                    print(name.lower())

sa.x, sa.y = m(sa.lon, sa.lat)

# Calculate emission
satotal = sum(sa.prod)
for xi, yi, sap in zip(sa.x, sa.y, sa.prod):
    xind=bisect.bisect_left(x0, xi)
    yind=bisect.bisect_left(y0, yi)
    if (yind<=mt.nrows)&(xind>=1):
        for n in range(5):
            sa.emi[n][yind-1][xind-1]+=sap/satotal*total_elec*(bd['AA'+index[n]].value+bd['AY'+index[n]].value)/10**6 * sa.df
np.save('./Data/sa', sa.emi)

In [ ]:
# Nitric acid emission

na = Emiss(df = 0.565)
naloc = []

for row in nacid.iter_rows('A2:G'+ str(nacid.max_row)):
    name = row[0].value
    if name != None:
        name = name.split(',')[0]
        name = name.strip()
        if 'total' not in name.lower():
            if name.lower() in fer_dict.keys():
                try:
                    na.prod.append(float(row[-1].value))
                    naloc.append(name)
                    lon, lat = fer_dict[name.lower()]
                    na.lon.append(lon)
                    na.lat.append(lat)
                except:
                    pass
                
na.x, na.y = m(na.lon, na.lat)

# Calculate emission
natotal = sum(na.prod)
for xi, yi, nap in zip(na.x, na.y, na.prod):
    xind=bisect.bisect_left(x0, xi)
    yind=bisect.bisect_left(y0, yi)
    if (yind<=mt.nrows)&(xind>=1):
        for n in range(5):
            na.emi[n][yind-1][xind-1]+=nap/natotal*total_elec*(bd['Y'+index[n]].value+bd['AW'+index[n]].value)/10**6 * na.df
np.save('./Data/na', na.emi)

In [ ]:
# Emission of Ammonium nitrate

anitr = Emiss(df = 0.565)
anitrloc = []

for row in an.iter_rows('A2:G'+ str(an.max_row)):
    name = row[0].value
    if name != None:
        name = name.split(',')[0]
        name = name.strip()
        if 'total' not in name.lower():
            if name.lower() in fer_dict.keys():
                try:
                    anitr.prod.append(float(row[-1].value))
#                     anitrloc.append(name)
                    lon, lat = fer_dict[name.lower()]
                    anitr.lon.append(lon)
                    anitr.lat.append(lat)
                except:
                    pass
                
anitr.x, anitr.y = m(anitr.lon, anitr.lat)

# Calculate emission
anitrtotal = sum(anitr.prod)
for xi, yi, anitrp in zip(anitr.x, anitr.y, anitr.prod):
    xind=bisect.bisect_left(x0, xi)
    yind=bisect.bisect_left(y0, yi)
    if (yind<=mt.nrows)&(xind>=1):
        for n in range(5):
            anitr.emi[n][yind-1][xind-1]+=anitrp/anitrtotal*total_elec*(bd['S'+index[n]].value+bd['AQ'+index[n]].value)/10**6 * anitr.df
np.save('./Data/anitr', anitr.emi)

In [ ]:
# Emission of Ammonia

am = Emiss(df = 0.565)
# anitrloc = []

for row in nh3.iter_rows('A2:G'+ str(nh3.max_row)):
    name = row[0].value
    if name != None:
        name = name.split(',')[0]
        name = name.strip()
        if 'total' not in name.lower():
            if name.lower() in fer_dict.keys():
                try:
                    am.prod.append(float(row[-1].value))
#                     anitrloc.append(name)
                    lon, lat = fer_dict[name.lower()]
                    am.lon.append(lon)
                    am.lat.append(lat)
                except:
                    pass
#             else:
#                 print(name.lower())
                
am.x, am.y = m(am.lon, am.lat)

# Calculate emission
amtotal = sum(am.prod)
for xi, yi, amp in zip(am.x, am.y, am.prod):
    xind=bisect.bisect_left(x0, xi)
    yind=bisect.bisect_left(y0, yi)
    if (yind<=mt.nrows)&(xind>=1):
        for n in range(5):
            am.emi[n][yind-1][xind-1]+=amp/amtotal*total_elec*(bd['Q'+index[n]].value+bd['AO'+index[n]].value+bd['X'+index[n]].value+bd['AV'+index[n]].value)/10**6 * am.df
np.save('./Data/am', am.emi)

In [ ]:
# Emission of P rock

pr = Emiss(df = 0.850)
# anitrloc = []

for row in prock.iter_rows('A2:G'+ str(prock.max_row)):
    name = row[0].value
    if name != None:
        name = name.split(',')[0]
        name = name.strip()
        if 'total' not in name.lower():
            if name.lower() in fer_dict.keys():
                try:
                    pr.prod.append(float(row[-1].value))
#                     anitrloc.append(name)
                    lon, lat = fer_dict[name.lower()]
                    pr.lon.append(lon)
                    pr.lat.append(lat)
                except:
                    pass
#             else:
#                 print(name.lower())
                
pr.x, pr.y = m(pr.lon, pr.lat)

# Calculate emission
prtotal = sum(pr.prod)
for xi, yi, prp in zip(pr.x, pr.y, pr.prod):
    xind=bisect.bisect_left(x0, xi)
    yind=bisect.bisect_left(y0, yi)
    if (yind<=mt.nrows)&(xind>=1) and (xind <= mt.ncols) and (yind >= 1):
        for n in range(5):
            pr.emi[n][yind-1][xind-1]+=prp/prtotal*total_elec*(bd['AB'+index[n]].value+bd['AZ'+index[n]].value)/10**6 * pr.df
np.save('./Data/pr', pr.emi)

In [ ]:
# Emission of P acid

pa = Emiss(df = 0.850)
# anitrloc = []

for row in pacid.iter_rows('A2:G'+ str(pacid.max_row)):
    name = row[0].value
    if name != None:
        name = name.split(',')[0]
        name = name.strip()
        if 'total' not in name.lower():
            if name.lower() in fer_dict.keys():
                try:
                    pa.prod.append(float(row[-1].value))
#                     anitrloc.append(name)
                    lon, lat = fer_dict[name.lower()]
                    pa.lon.append(lon)
                    pa.lat.append(lat)
                except:
                    pass
#             else:
#                 print(name.lower())
                
pa.x, pa.y = m(pa.lon, pa.lat)

# Calculate emission
patotal = sum(pa.prod)
for xi, yi, pap in zip(pa.x, pa.y, pa.prod):
    xind=bisect.bisect_left(x0, xi)
    yind=bisect.bisect_left(y0, yi)
    if (yind<=mt.nrows)&(xind>=1) and (xind <= mt.ncols) and (yind >= 1):
        for n in range(5):
            pa.emi[n][yind-1][xind-1]+=pap/patotal*total_elec*(bd['Z'+index[n]].value+bd['AX'+index[n]].value)/10**6 * pa.df
np.save('./Data/pa', pa.emi)

In [ ]:
# Emission of UAN

uan = Emiss(df = 0.565)

for row in usolution.iter_rows('A2:G'+ str(usolution.max_row)):
    name = row[0].value
    if name != None:
        name = name.split(',')[0]
        name = name.strip()
        if 'total' not in name.lower():
            if name.lower() in fer_dict.keys():
                try:
                    uan.prod.append(float(row[-1].value))
#                     anitrloc.append(name)
                    lon, lat = fer_dict[name.lower()]
                    uan.lon.append(lon)
                    uan.lat.append(lat)
                except:
                    pass
#             else:
#                 print(name.lower())
                
uan.x, uan.y = m(uan.lon, uan.lat)

# Calculate emission
uantotal = sum(uan.prod)
for xi, yi, uanp in zip(uan.x, uan.y, uan.prod):
    xind=bisect.bisect_left(x0, xi)
    yind=bisect.bisect_left(y0, yi)
    if (yind<=mt.nrows)&(xind>=1) and (xind <= mt.ncols) and (yind >= 1):
        for n in range(5):
            uan.emi[n][yind-1][xind-1]+=uanp/uantotal*total_elec*(bd['V'+index[n]].value+bd['AT'+index[n]].value)/10**6 * uan.df
np.save('./Data/uan', uan.emi)

In [ ]:
# Emission of Ammonium sulfate

asu = Emiss(df = 0.565)

for row in asul.iter_rows('A2:G'+ str(asul.max_row)):
    name = row[0].value
    if name != None:
        name = name.split(',')[0]
        name = name.strip()
        if 'total' not in name.lower():
            if name.lower() in fer_dict.keys():
                try:
                    asu.prod.append(float(row[-1].value))
#                     anitrloc.append(name)
                    lon, lat = fer_dict[name.lower()]
                    asu.lon.append(lon)
                    asu.lat.append(lat)
                except:
                    pass
#             else:
#                 print(name.lower())
                
asu.x, asu.y = m(asu.lon, asu.lat)

# Calculate emission
asutotal = sum(asu.prod)
for xi, yi, asup in zip(asu.x, asu.y, asu.prod):
    xind=bisect.bisect_left(x0, xi)
    yind=bisect.bisect_left(y0, yi)
    if (yind<=mt.nrows)&(xind>=1) and (xind <= mt.ncols) and (yind >= 1):
        for n in range(5):
            asu.emi[n][yind-1][xind-1]+=asup/asutotal*total_elec*(bd['W'+index[n]].value+bd['AU'+index[n]].value)/10**6 * asu.df
np.save('./Data/asu', asu.emi)

In [ ]:
# Emission of DAP & MAP

dm = Emiss(df = 0.565)

for row in dmap.iter_rows('A2:G'+ str(dmap.max_row)):
    name = row[0].value
    if name != None:
        name = name.split(',')[0]
        name = name.strip()
        if 'total' not in name.lower():
            if name.lower() in fer_dict.keys():
                try:
                    dm.prod.append(float(row[-1].value))
#                     anitrloc.append(name)
                    lon, lat = fer_dict[name.lower()]
                    dm.lon.append(lon)
                    dm.lat.append(lat)
                except:
                    pass
#             else:
#                 print(name.lower())
                
dm.x, dm.y = m(dm.lon, dm.lat)

# Calculate emission
dmtotal = sum(dm.prod)
for xi, yi, dmp in zip(dm.x, dm.y, dm.prod):
    xind=bisect.bisect_left(x0, xi)
    yind=bisect.bisect_left(y0, yi)
    if (yind<=mt.nrows)&(xind>=1) and (xind <= mt.ncols) and (yind >= 1):
        for n in range(5):
            dm.emi[n][yind-1][xind-1]+=dmp/dmtotal*total_elec*(bd['T'+index[n]].value+bd['U'+index[n]].value+bd['AR'+index[n]].value+bd['AS'+index[n]].value)/10**6 * dm.df
np.save('./Data/dm', dm.emi)

In [ ]:
# Coal mine emissions

cmine = Emiss()

wbc = load_workbook('/Users/oulongwen/Downloads/coal mines.xlsx',data_only=True, read_only = True)
wsc = wbc['2008Pub']
fip = []

for row in wsc.iter_rows('M2:Y'+str(wsc.max_row)):
        cmine.prod.append(row[-4].value)
        cmine.lat.append(row[-2].value)
        cmine.lon.append(row[-1].value)
        fip.append(row[0].value)

cmine.x,cmine.y = m(cmine.lon,cmine.lat)

# Calculate emission
cminetotal = sum(cmine.prod)
for xi, yi, cminep in zip(cmine.x, cmine.y, cmine.prod):
    xind=bisect.bisect_left(x0, xi)
    yind=bisect.bisect_left(y0, yi)
    if (yind<=mt.nrows)&(xind>=1) and (xind <= mt.ncols) and (yind >= 1):
        for n in range(5):
            cmine.emi[n][yind-1][xind-1]+=cminep/cminetotal*total_elec*bd['BM'+index[n]].value/10**6 * cmine.df
np.save('./Data/cmine', cmine.emi)
m.scatter(cmine.x,cmine.y, marker = 'o', s = np.array(cmine.prod)/cminetotal * 1000)
plt.show()

# State mine
state = pd.read_csv('/Users/oulongwen/Downloads/state_latlon.csv',index_col = 'FIPS')
state['prod'] = pd.Series(np.zeros(len(state.index)),index = state.index)
for id, prod in zip(fip, cmine.prod):
    state.ix[int(id),'prod'] += prod

state = state.drop(state.index[[0,11]])     # remove HI and AK
statecmine = Emiss()
statecmine.lon = np.array(state['longitude'])
statecmine.lat = np.array(state['latitude'])
statecmine.prod = np.array(state['prod'])
statecmine.x, statecmine.y = m(statecmine.lon, statecmine.lat)

In [5]:
# Emission of Coal and NG power plants

coale = Emiss()
nge = Emiss()
coalfip = []
ngfip = []

wbp = load_workbook('/Users/oulongwen/Google Drive/self share/power plants.xlsx',data_only=True, read_only = True)
wsp = wbp['power']
for row in wsp.iter_rows('E2:U'+str(wsp.max_row)):
    if row[3].value > 0:
        coale.prod.append(row[3].value)
        coale.lon.append(row[-1].value)
        coale.lat.append(row[-2].value)
        coalfip.append(row[0].value)
    if row[4].value > 0:
        nge.prod.append(row[4].value)
        nge.lon.append(row[-1].value)
        nge.lat.append(row[-2].value)
        ngfip.append(row[0].value)

# Calculate emission for coal power plants
coale.x,coale.y = m(coale.lon,coale.lat)
coaletotal = sum(coale.prod)
for xi, yi, coalep in zip(coale.x, coale.y, coale.prod):
    xind=bisect.bisect_left(x0, xi)
    yind=bisect.bisect_left(y0, yi)
    if (yind<=mt.nrows)&(xind>=1) and (xind <= mt.ncols) and (yind >= 1):
        for n in range(5):
            coale.emi[n][yind-1][xind-1]+=coalep/coaletotal*total_elec*bd['BO'+index[n]].value/10**6 * coale.df
np.save('./Data/coale', coale.emi)


# State corn production
state = pd.read_csv('/Users/oulongwen/Downloads/state_latlon.csv',index_col = 'FIPS')
state['prod'] = pd.Series(np.zeros(len(state.index)),index = state.index)

for id, production in zip(coalfip, coale.prod):
    state.ix[int(id),'prod'] += production
state = state.drop(state.index[[0,11]])     # remove HI and AK
statecoale = Emiss()
statecoale.lon = np.array(state['longitude'])
statecoale.lat = np.array(state['latitude'])
statecoale.prod = np.array(state['prod'])
statecoale.x, statecoale.y = m(statecoale.lon, statecoale.lat)
        
# Calculate emission for NG power plants
nge.x,nge.y = m(nge.lon,nge.lat)
ngetotal = sum(nge.prod)
for xi, yi, ngep in zip(nge.x, nge.y, nge.prod):
    xind=bisect.bisect_left(x0, xi)
    yind=bisect.bisect_left(y0, yi)
    if (yind<=mt.nrows)&(xind>=1) and (xind <= mt.ncols) and (yind >= 1):
        for n in range(5):
            nge.emi[n][yind-1][xind-1]+=ngep/ngetotal*total_elec*bd['BP'+index[n]].value/10**6 * nge.df
np.save('./Data/nge', nge.emi)

In [9]:
len(coale.x)

454

In [14]:
# Emission of transportation of coal

coaltr = Emiss()
# res = mt.transemi(cmine, coale, 500, 100)
res = mt.transemi(statecmine, statecoale, len(statecmine.x), len(statecoale.x))
for n in range(5):
    coaltr.emi[n] += res / mt.totaltrans * total_elec * bd['BN'+index[n]].value / 10**6 * coaltr.df
np.save('./Data/coaltr', coaltr.emi)

NameError: name 'statecmne' is not defined

In [6]:
# Emission of transportation of corn stover

stovertr = Emiss()
# res = mt.transemi(corn, coale, 600, 120)
res = mt.transemi(statecorn, statecoale, len(statecorn.x), len(statecoale.x))
for n in range(5):
    stovertr.emi[n] += res / mt.totaltrans * total_elec * bd['J'+index[n]].value / 10**6 * stovertr.df
# np.save('./Data/stovertr', stovertr.emi)

In [29]:
np.save('./Data/fig_sols', np.array(mt.sols['x']))
np.save('./Data/fig_paths', np.array(mt.paths))
np.save('./Data/fig_d',np.array(mt.d))

In [8]:
-
# plt.show()

In [6]:
max(fretemp)

2175436504.2014027

In [78]:
# Emission of transportation of urea for N

ureatr = Emiss(df = 0.565)
# res = mt.transemi(u, corn, len(u.x), 1200)
res = mt.transemi(u, statecorn, len(u.x), len(statecorn.x))
for n in range(5):
    ureatr.emi[n] += res / mt.totaltrans * total_elec * bd['AD'+index[n]].value / 10**6 * ureatr.df
np.save('./Data/ureatr', ureatr.emi)

In [80]:
len(statecorn.x)

49

In [82]:
# Emission of transportation of ammonium nitrate for N

anitrtr = Emiss(df = 0.565)
# res = mt.transemi(anitr, corn, len(anitr.x), 1200)
res = mt.transemi(anitr, statecorn, len(anitr.x), len(statecorn.x))
for n in range(5):
    anitrtr.emi[n] += res / mt.totaltrans * total_elec * bd['AE'+index[n]].value / 10**6 * anitrtr.df
np.save('./Data/anitrtr', anitrtr.emi)

In [83]:
anitrtr.emi.sum()

604.39382288626189

In [84]:
# Emission of transportation of ammonia for N as final fertilizer

amtr = Emiss(df = 0.565)
res = mt.transemi(am, statecorn, len(am.x), len(statecorn.x))
# res = mt.transemi(am, corn, len(am.x), 1200)
for n in range(5):
    amtr.emi[n] += res / mt.totaltrans * total_elec * bd['AC'+index[n]].value / 10**6 * amtr.df
np.save('./Data/amtr', amtr.emi)

In [85]:
# Emission of transportation of DAP & MAP for N and P total

dmaptr = Emiss(df = 0.565)
# res = mt.transemi(dm, corn, len(dm.x), 1200)
res = mt.transemi(dm, statecorn, len(dm.x), len(statecorn.x))
for n in range(5):
    dmaptr.emi[n] += res / mt.totaltrans * total_elec * (bd['AF'+index[n]].value+bd['AG'+index[n]].value+bd['BD'+index[n]].value+bd['BE'+index[n]].value) / 10**6 * dmaptr.df
np.save('./Data/dmaptr', dmaptr.emi)

In [86]:
# Emission of transportation of UAN

uantr = Emiss(df = 0.565)
# res = mt.transemi(uan, corn, len(uan.x), 1200)
res = mt.transemi(uan, statecorn, len(uan.x), len(statecorn.x))
for n in range(5):
    uantr.emi[n] += res / mt.totaltrans * total_elec * (bd['AH'+index[n]].value + bd['BF'+index[n]].value) / 10**6 * uantr.df
np.save('./Data/uantr', uantr.emi)

In [87]:
# Emission of transportation of ammonium sulfate

asutr = Emiss(df = 0.565)
# res = mt.transemi(asu, corn, len(asu.x), 1200)
res = mt.transemi(asu, statecorn, len(asu.x), len(statecorn.x))
for n in range(5):
    asutr.emi[n] += res / mt.totaltrans * total_elec * (bd['AI'+index[n]].value + bd['BG'+index[n]].value) / 10**6 * asutr.df 
np.save('./Data/asutr', asutr.emi)

In [88]:
# Emission of transportation of sulfuric acid

ancillary = Emiss()
scale = bdwb['N-2']
scasul = scale['G40'].value
scpa = scale['H40'].value
ancillary.x = asu.x + pa.x
ancillary.y = asu.y + pa.y
ancillary.prod = [i*scasul for i in asu.prod] + [i*scpa for i in pa.prod]
ancillary.lon = asu.lon + pa.lon
ancillary.lat = asu.lat + pa.lat
satr = Emiss(df = 0.720)
res = mt.transemi(sa, ancillary, len(sa.x), len(ancillary.x))
for n in range(5):
    satr.emi[n] += res / mt.totaltrans * total_elec * (bd['AM'+index[n]].value + bd['BK'+index[n]].value) / 10**6 * satr.df
np.save('./Data/satr', satr.emi)

In [38]:
len(sa.lon)

67

In [31]:
# Emission of transportation of P acid

patr = Emiss(df = 0.850)
res = mt.transemi(pa, dm, len(pa.x), len(dm.x))
for n in range(5):
    patr.emi[n] += res / mt.totaltrans * total_elec * (bd['AL'+index[n]].value + bd['BJ'+index[n]].value) / 10**6 * patr.df
np.save('./Data/patr', patr.emi)

In [32]:
# Emission of transportation of P rock

prtr = Emiss(df = 0.850)
res = mt.transemi(pr, pa, len(pr.x), len(pa.x))
for n in range(5):
    prtr.emi[n] += res / mt.totaltrans * total_elec * (bd['AN'+index[n]].value + bd['BL'+index[n]].value) / 10**6 * prtr.df
np.save('./Data/prtr', prtr.emi)

In [33]:
# Emission of transportation of N acid

natr = Emiss(df = 0.565)
res = mt.transemi(na, anitr, len(na.x), len(anitr.x))
for n in range(5):
    natr.emi[n] += res / mt.totaltrans * total_elec * (bd['AK'+index[n]].value + bd['BI'+index[n]].value) / 10**6 * natr.df
np.save('./Data/natr', natr.emi)

In [32]:
# Emission of transportation of K2O

ktr = Emiss(df = 0.170)
# res = mt.transemi(k, corn, len(k.x), 1200)
res = mt.transemi(k, statecorn, len(k.x), len(statecorn.x))
for n in range(5):
    ktr.emi[n] += res / mt.totaltrans * total_elec * bd['M'+index[n]].value / 10**6 * ktr.df
np.save('./Data/ktr', ktr.emi)

In [43]:
# Emission of transportation of intermediate ammonia

ancillary2 = Emiss()
scale = bdwb['N-2']
scurea = scale['J38'].value
scan = scale['K38'].value
scdm = scale['N38'].value
scasu = scale['O38'].value
scna = scale['P38'].value
ancillary2.x = u.x + anitr.x + dm.x + asu.x + na.x
ancillary2.y = u.y + anitr.y + dm.y + asu.y + na.y
ancillary2.prod = [i*scurea for i in u.prod] + [i*scan for i in anitr.prod] + [i*scdm for i in dm.prod] + [i*scasu for i in asu.prod] + [i*scna for i in na.prod]
ancillary2.lon = u.lon + anitr.lon + dm.lon + asu.lon + na.lon
ancillary2.lat = u.lat + anitr.lat + dm.lat + asu.lat + na.lat
iamtr = Emiss(df = 0.565)
res = mt.transemi(am, ancillary2, len(am.x), len(ancillary2.x))
for n in range(5):
    iamtr.emi[n] += res / mt.totaltrans * total_elec * (bd['AJ'+index[n]].value + bd['BH'+index[n]].value) / 10**6 * iamtr.df
np.save('./Data/iamtr', iamtr.emi)

In [42]:
len(u.x + anitr.x + dm.x + asu.x + na.x)
len(am.x)

23

In [45]:
iamtr.emi[0].sum()

28.952638048761226

In [48]:
len(uan.x)

19